In [1]:
from datashop import*
from pprint import pprint as pp
import requests
import math
import time
import json

key = 'm6HsQpjurALl27VpNGB1o4nAcr50o5wt'

response = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key={}'.format(key)


In [4]:
def get_docs(start_date,end_date):

    params ={
        'begin_date':start_date,
        'end_date':end_date,
        'q':'Oil (Petroleum) and Gasoline',
    }
    request = requests.get(response,params)
    series_dict = json.loads(request.text)

    meta = series_dict['response']['meta']

    pages = math.ceil(meta['hits']/10)

    collection = []

    for page in range(0,pages):

        params['page']= page

        request = requests.get(response,params)
        series_dict = json.loads(request.text)

        docs = series_dict['response']['docs']

        for doc in docs:
            collection.append(doc)

        time.sleep(7)

    with open('nyt_jsons/' + str(start_date) + '_articles.json', 'w') as f:
        json.dump(collection, f)    

In [7]:
years = ['20' + str(x).zfill(2) for x in range(6,21)]

for year in years:
    start = year + '0101'
    end = year + '1231'
    collection = get_docs(start,end)
    print(year + " Has been saved")

2006 Has been saved
2007 Has been saved
2008 Has been saved
2009 Has been saved
2010 Has been saved
2011 Has been saved
2012 Has been saved
2013 Has been saved
2014 Has been saved
2015 Has been saved
2016 Has been saved
2017 Has been saved
2018 Has been saved
2019 Has been saved
2020 Has been saved


In [38]:
def parse_docs(docs):
    doc_list = []

    for doc in docs:
        doc_dict = {
            'id':doc['_id'],
            'date':doc['pub_date'],
            'abstract':doc['abstract'],
            'doc_type':doc['document_type'],
            'main_headline':doc['headline']['main'],
            'print_headline':doc['headline']['print_headline'],
            'keywords':doc['keywords'],
            'newsdesk':doc['news_desk'],
            
            

        }

        doc_list.append(doc_dict)

    frame = pd.DataFrame(doc_list)
    return frame    

In [40]:
import os

art_list = []

for file in os.listdir('nyt_jsons'):
    with open('nyt_jsons/' + file) as json_file:
        data = json.load(json_file)
        art_list =art_list + data

final_frame = parse_docs(art_list)
final_frame.to_csv('data/nyt_articles.csv')

In [30]:
display(final_frame)

,id,date,abstract,doc_type,main_headline,print_headline,keywords
0,nyt://article/0914c010-16d4-54bf-9098-4045adc3...,2000-12-30T05:00:00+0000,Berkley Petroleum Corp will consider takeover ...,article,BERKLEY PETROLEUM APPROVES SHAREHOLDER RIGHTS ...,COMPANY NEWS; BERKLEY PETROLEUM APPROVES SHARE...,"[{'name': 'organizations', 'value': 'Hunt Oil ..."
1,nyt://article/1fe65aa7-59d3-500d-8071-b32e5eb5...,2000-12-23T05:00:00+0000,Delta Petroleum Corp cancels agreement to buy ...,article,Delta Petroleum in Deal,Delta Petroleum in Deal,"[{'name': 'glocations', 'value': 'New Mexico',..."
2,nyt://article/e084f9f3-03e1-5608-a6ec-923d6bd3...,2000-12-29T05:00:00+0000,Heating oil futures fall after report showing ...,article,THE MARKETS: COMMODITIES,THE MARKETS: COMMODITIES,"[{'name': 'subject', 'value': 'Oil (Petroleum)..."
3,nyt://article/07ee3d9c-57c2-5e1c-b1cc-b4b1cbbe...,2000-12-31T05:00:00+0000,"Petrobras, Brazil's state-controlled oil compa...",article,Exploratory Oil Drilling From S to X and Back,BUSINESS: DIARY; Exploratory Oil Drilling From...,"[{'name': 'glocations', 'value': 'Brazil', 'ra..."
4,nyt://article/f577789f-abf5-5275-be64-cd1fe8fd...,2000-12-27T05:00:00+0000,Heating oil prices for January delivery soar m...,article,THE MARKETS: COMMODITIES,THE MARKETS: COMMODITIES,"[{'name': 'subject', 'value': 'Weather', 'rank..."
...,...,...,...,...,...,...,...
12478,nyt://interactive/768a0a5d-1c61-52d2-b379-4f92...,2020-04-20T01:36:12+0000,Another installment of our climate crash cours...,multimedia,Do environmental rules matter?,None,[]
12479,nyt://interactive/821f839b-caf1-58e2-b200-26be...,2020-04-20T01:37:46+0000,"Hiroko Tabuchi, an investigative reporter for ...",multimedia,Who is influencing key decisions?,None,[]
12480,nyt://article/1904f2c1-1839-5d86-b106-1a807f02...,2020-04-23T04:01:33+0000,Britain shrouds science advice in secrecy. Pak...,article,"In Ecuador, Coronavirus Deaths Go Uncounted",None,"[{'name': 'subject', 'value': 'Coronavirus (20..."
12481,nyt://article/23b94cb1-266f-5288-bbe4-55a27a7a...,2020-04-04T04:02:19+0000,President Trump warns of what could be “the to...,article,Trump Warns of ‘a Lot of Death’ While Also Rev...,Around The U.S.,"[{'name': 'subject', 'value': 'Coronavirus (20..."


In [31]:
final_frame.to_csv('data/nyt_articles.csv')